In [19]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns


dfCounty = pd.read_csv("data/us_county.csv")
dfPoverty = pd.read_csv("data/PovertyEstimates_us_county.csv")
dfcovCnty = pd.read_csv("data/covid_us_county.csv")
dfvaccine = pd.read_csv("data/us_state_vaccinations.csv")


NameError: name 'head' is not defined

In [21]:
print("US County")
display(dfCounty.head())
display(dfPoverty.head())
display(dfcovCnty.head())
display(dfvaccine.head())

,fips,county,state,state_code,male,female,median_age,population,female_percentage,lat,long
0,1001,Autauga County,Alabama,AL,26874,28326,37.8,55200,51.315217,32.534923,-86.642730
1,1003,Baldwin County,Alabama,AL,101188,106919,42.8,208107,51.376936,30.727479,-87.722564
2,1005,Barbour County,Alabama,AL,13697,12085,39.9,25782,46.873788,31.869581,-85.393210
3,1007,Bibb County,Alabama,AL,12152,10375,39.9,22527,46.055844,32.998628,-87.126475
4,1009,Blount County,Alabama,AL,28434,29211,40.8,57645,50.673953,33.980869,-86.567380


,FIPStxt,Stabr,Area_name,Rural-urban_Continuum_Code_2003,Urban_Influence_Code_2003,Rural-urban_Continuum_Code_2013,Urban_Influence_Code_2013,POVALL_2018,CI90LBAll_2018,CI90UBALL_2018,...,CI90UB517P_2018,MEDHHINC_2018,CI90LBINC_2018,CI90UBINC_2018,POV04_2018,CI90LB04_2018,CI90UB04_2018,PCTPOV04_2018,CI90LB04P_2018,CI90UB04P_2018
0,0,US,United States,NaN,NaN,NaN,NaN,41852315,41619366,42085264,...,17.2,61937,61843,62031,3758704.0,3714862.0,3802546.0,19.5,19.3,19.7
1,1000,AL,Alabama,NaN,NaN,NaN,NaN,801758,785668,817848,...,23.7,49881,49123,50639,73915.0,69990.0,77840.0,26.0,24.6,27.4
2,1001,AL,Autauga County,2.0,2.0,2.0,2.0,7587,6334,8840,...,23.9,59338,53628,65048,NaN,NaN,NaN,NaN,NaN,NaN
3,1003,AL,Baldwin County,4.0,5.0,3.0,2.0,21069,17390,24748,...,16.9,57588,54437,60739,NaN,NaN,NaN,NaN,NaN,NaN
4,1005,AL,Barbour County,6.0,6.0,6.0,6.0,6788,5662,7914,...,45.9,34382,31157,37607,NaN,NaN,NaN,NaN,NaN,NaN


,fips,county,state,lat,long,date,cases,state_code,deaths
0,1001.0,Autauga,Alabama,32.539527,-86.644082,2020-01-22,0,AL,0
1,1001.0,Autauga,Alabama,32.539527,-86.644082,2020-01-23,0,AL,0
2,1001.0,Autauga,Alabama,32.539527,-86.644082,2020-01-24,0,AL,0
3,1001.0,Autauga,Alabama,32.539527,-86.644082,2020-01-25,0,AL,0
4,1001.0,Autauga,Alabama,32.539527,-86.644082,2020-01-26,0,AL,0


,date,location,total_vaccinations,total_distributed,people_vaccinated,people_fully_vaccinated_per_hundred,total_vaccinations_per_hundred,people_fully_vaccinated,people_vaccinated_per_hundred,distributed_per_hundred,daily_vaccinations_raw,daily_vaccinations,daily_vaccinations_per_million,share_doses_used
0,2021-01-12,Alabama,78134.0,377025.0,70861.0,0.15,1.59,7270.0,1.45,7.69,NaN,NaN,NaN,0.207
1,2021-01-13,Alabama,84040.0,378975.0,74792.0,0.19,1.71,9245.0,1.53,7.73,5906.0,5906.0,1205.0,0.222
2,2021-01-14,Alabama,92300.0,435350.0,80480.0,NaN,1.88,NaN,1.64,8.88,8260.0,7083.0,1445.0,0.212
3,2021-01-15,Alabama,100567.0,444650.0,86956.0,0.28,2.05,13488.0,1.77,9.07,8267.0,7478.0,1525.0,0.226
4,2021-01-16,Alabama,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7557.0,7498.0,1529.0,NaN


In [ ]:
dfPoverty1 = dfPoverty.drop(['Rural-urban_Continuum_Code_2003',
                             'Urban_Influence_Code_2003',
                             'POV04_2018', 'CI90LB04_2018', 'CI90UB04_2018', 'PCTPOV04_2018',
                             'CI90LB04P_2018', 'CI90UB04P_2018'], axis = 1)

In [52]:
dfPoverty.columns

Index(['FIPStxt', 'Stabr', 'Area_name', 'Rural-urban_Continuum_Code_2003',
       'Urban_Influence_Code_2003', 'Rural-urban_Continuum_Code_2013',
       'Urban_Influence_Code_2013', 'POVALL_2018', 'CI90LBAll_2018',
       'CI90UBALL_2018', 'PCTPOVALL_2018', 'CI90LBALLP_2018',
       'CI90UBALLP_2018', 'POV017_2018', 'CI90LB017_2018', 'CI90UB017_2018',
       'PCTPOV017_2018', 'CI90LB017P_2018', 'CI90UB017P_2018', 'POV517_2018',
       'CI90LB517_2018', 'CI90UB517_2018', 'PCTPOV517_2018', 'CI90LB517P_2018',
       'CI90UB517P_2018', 'MEDHHINC_2018', 'CI90LBINC_2018', 'CI90UBINC_2018',
       'POV04_2018', 'CI90LB04_2018', 'CI90UB04_2018', 'PCTPOV04_2018',
       'CI90LB04P_2018', 'CI90UB04P_2018'],
      dtype='object')

In [23]:
dfCounty.describe()

,fips,male,female,median_age,population,female_percentage,lat,long
count,3220.000000,3.220000e+03,3.220000e+03,3220.000000,3.220000e+03,3220.000000,3220.000000,3220.000000
mean,31393.605280,4.987486e+04,5.145743e+04,41.277516,1.013323e+05,49.961246,37.966287,-91.641071
std,16292.078954,1.601656e+05,1.659688e+05,5.356761,3.260964e+05,2.374927,6.095580,13.311311
min,1001.000000,3.800000e+01,3.700000e+01,21.700000,7.500000e+01,21.003945,17.981964,-164.027212
25%,19032.500000,5.657500e+03,5.573000e+03,38.100000,1.121425e+04,49.433348,34.353322,-98.086128
50%,30024.000000,1.291650e+04,1.299650e+04,41.200000,2.595050e+04,50.415031,38.212520,-89.945290
75%,46105.500000,3.324850e+04,3.353075e+04,44.300000,6.655225e+04,51.149548,41.694910,-82.992494
max,72153.000000,4.976788e+06,5.121264e+06,67.000000,1.009805e+07,58.613660,69.312150,-65.286183


In [24]:
dfPoverty.describe()

,FIPStxt,Rural-urban_Continuum_Code_2003,Urban_Influence_Code_2003,Rural-urban_Continuum_Code_2013,Urban_Influence_Code_2013,POVALL_2018,CI90LBAll_2018,CI90UBALL_2018,PCTPOVALL_2018,CI90LBALLP_2018,...,CI90UB517P_2018,MEDHHINC_2018,CI90LBINC_2018,CI90UBINC_2018,POV04_2018,CI90LB04_2018,CI90UB04_2018,PCTPOV04_2018,CI90LB04P_2018,CI90UB04P_2018
count,3193.000000,3136.000000,3136.000000,3141.000000,3141.000000,3.193000e+03,3.193000e+03,3.193000e+03,3193.000000,3193.000000,...,3193.000000,3193.000000,3193.000000,3193.00000,5.200000e+01,5.200000e+01,5.200000e+01,52.000000,52.000000,52.000000
mean,30356.223301,5.124362,5.447704,5.009233,5.268704,3.932257e+04,3.752753e+04,4.111761e+04,15.125274,12.277513,...,25.308675,52944.895709,48822.994989,57066.79643,1.445656e+05,1.397077e+05,1.494235e+05,18.890385,17.446154,20.334615
std,15178.992102,2.681944,3.469504,2.708605,3.499373,7.583740e+05,7.538297e+05,7.629268e+05,6.092508,4.995706,...,11.075640,13878.755803,13616.805496,14259.40251,5.188033e+05,5.128637e+05,5.247517e+05,5.009648,4.975409,5.096900
min,0.000000,1.000000,1.000000,1.000000,1.000000,5.000000e+00,4.000000e+00,6.000000e+00,2.600000,2.100000,...,2.800000,25385.000000,22961.000000,27809.00000,3.420000e+03,2.872000e+03,3.968000e+03,10.600000,9.400000,11.800000
25%,18173.000000,3.000000,2.000000,2.000000,2.000000,1.621000e+03,1.259000e+03,1.969000e+03,10.800000,8.700000,...,17.100000,43783.000000,39863.000000,47752.00000,1.932450e+04,1.743400e+04,2.121500e+04,15.000000,13.575000,16.200000
50%,29175.000000,6.000000,5.000000,6.000000,5.000000,3.876000e+03,3.091000e+03,4.708000e+03,14.100000,11.400000,...,23.800000,50748.000000,46553.000000,54667.00000,4.844150e+04,4.474050e+04,5.214250e+04,18.400000,16.800000,20.000000
75%,45075.000000,7.000000,8.000000,7.000000,8.000000,9.874000e+03,8.254000e+03,1.158400e+04,18.300000,14.900000,...,31.600000,59020.000000,54639.000000,63295.00000,8.959875e+04,8.452250e+04,9.467500e+04,22.475000,21.025000,23.550000
max,56045.000000,9.000000,12.000000,9.000000,12.000000,4.185232e+07,4.161937e+07,4.208526e+07,54.000000,47.400000,...,80.500000,140382.000000,135638.000000,150364.00000,3.758704e+06,3.714862e+06,3.802546e+06,30.800000,29.000000,32.600000


In [49]:
state_pop = pd.DataFrame(dfCounty.groupby('state')['male','female','population'].sum())
state_pop

<ipython-input-49-55b33c348073>:1: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  state_pop = pd.DataFrame(dfCounty.groupby('state')['male','female','population'].sum())


,male,female,population
state,,,
Alabama,2355799,2508881,4864680
Alaska,385579,352937,738516
Arizona,3453439,3493246,6946685
Arkansas,1468412,1522259,2990671
California,19453769,19694991,39148760
Colorado,2780831,2750310,5531141
Connecticut,1747131,1834373,3581504
Delaware,459570,489925,949495
District of Columbia,324881,359617,684498


In [43]:
pd.DataFrame(dfPoverty['Stabr'].unique())

,0
0,US
1,AL
2,AK
3,AZ
4,AR
5,CA
6,CO
7,CT
8,DE
9,DC


In [44]:
dfCounty[dfCounty["state"] == 'Puerto Rico']

,fips,county,state,state_code,male,female,median_age,population,female_percentage,lat,long
3142,72001,Adjuntas Municipio,Puerto Rico,NaN,8862,9319,40.9,18181,51.256807,18.179734,-66.753959
3143,72003,Aguada Municipio,Puerto Rico,NaN,18906,19737,42.3,38643,51.075227,18.360531,-67.174999
3144,72005,Aguadilla Municipio,Puerto Rico,NaN,26308,27858,41.5,54166,51.430787,18.459736,-67.120762
3145,72007,Aguas Buenas Municipio,Puerto Rico,NaN,12756,13519,41.4,26275,51.451951,18.251057,-66.127337
3146,72009,Aibonito Municipio,Puerto Rico,NaN,11263,12194,41.6,23457,51.984482,18.131120,-66.264417
...,...,...,...,...,...,...,...,...,...,...,...
3215,72145,Vega Baja Municipio,Puerto Rico,NaN,25580,27791,40.7,53371,52.071350,18.428461,-66.397926
3216,72147,Vieques Municipio,Puerto Rico,NaN,4332,4439,43.6,8771,50.609965,18.122662,-65.439095
3217,72149,Villalba Municipio,Puerto Rico,NaN,11169,11824,38.8,22993,51.424347,18.128155,-66.472816
3218,72151,Yabucoa Municipio,Puerto Rico,NaN,16541,17608,42.5,34149,51.562271,18.070468,-65.896311
